# 自然言語処理によるトピック分類
自然言語処理（以下NLP）により、トピックの分類は可能であるが、どの程度の精度で分類が可能であるか実験をしてみる。
使用するデータは、クリエイティブ・コモンズライセンスのもとに提供されている「Livedoor News」を使用し、8トピック、6600文章程度のデータセットとなっている。

＜Livedoor News＞
- トピックニュース
- Sports Watch
- ITライフハック
- 家電チャンネル
- MOVIE ENTER
- 独女通信
- エスマックス
- livedoor HOMME
- Peachy

## 1.データセットの読込み

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import time
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format

In [2]:
#https://qiita.com/hyo_07/items/ba3d53868b2f55ed9941
import os

#ファイルの読み込み、及びタイトル・本文データの取得
path_list = ["dokujo-tsushin", "it-life-hack", "movie-enter","sports-watch","kaden-channel","livedoor-homme","peachy","smax"]

w_list = []
labels = []

for p_list in path_list:
    path = "./text/"+p_list
    #ディレクトリ内の全ファイル名を取得
    f_list = os.listdir(path)

    for lists in f_list:
        with open("./text/"+ p_list+ "/"+lists, encoding="utf-8_sig") as f:
            next(f)
            next(f)
            #全角スペースや改行の削除
            w = f.read().replace('\u3000','').replace('\n','')

            w_list.append(w)
            labels.append(path_list.index(p_list))

In [3]:
alldata=pd.DataFrame({"text":w_list,"label":labels})
print("alldata.shape:",alldata.shape)
print("alldata['label'].unique():",alldata['label'].unique())
alldata

alldata.shape: (6605, 2)
alldata['label'].unique(): [0 1 2 3 4 5 6 7]


,text,label
0,友人代表のスピーチ、独女はどうこなしている？もうすぐジューン・ブライドと呼ばれる６月。独女の...,0
1,ネットで断ち切れない元カレとの縁携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だ...,0
2,相次ぐ芸能人の“すっぴん”披露その時、独女の心境は？「男性はやっぱり、女性の“すっぴん”が大...,0
3,ムダな抵抗！？ 加齢の現実ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「...,0
4,税金を払うのは私たちなんですけど！6月から支給される子ども手当だが、当初は子ども一人当たり月...,0
...,...,...
6600,キレイに撮れる！docomoのスマホ「GALAXY S III SC-06D」のカメラでE2...,7
6601,Sony Mobile、4.6インチディスプレイを搭載したスマートフォン「Xperia T」...,7
6602,タブレットもXperiaブランドへ！ソニー、9.4インチAndroidタブレット「Xperi...,7
6603,かわいいから大人買いしたい！邪魔なケーブルをすっきりできる「ドロイド君 ケーブルホルダー」【...,7


In [4]:
alldata_original=alldata.copy()

In [5]:
alldata_original.loc[0,'text']

'友人代表のスピーチ、独女はどうこなしている？もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり……という「お祝い貧乏」状態の人も多いのではないだろうか？さらに出席回数を重ねていくと、こんなお願いごとをされることも少なくない。「お願いがあるんだけど……友人代表のスピーチ、やってくれないかな？」さてそんなとき、独女はどう対応したらいいか？最近だとインターネット等で検索すれば友人代表スピーチ用の例文サイトがたくさん出てくるので、それらを参考にすれば、無難なものは誰でも作成できる。しかし由利さん（33歳）はネットを参考にして作成したものの「これで本当にいいのか不安でした。一人暮らしなので聞かせて感想をいってくれる人もいないし、かといって他の友人にわざわざ聞かせるのもどうかと思うし……」ということで活用したのが、なんとインターネットの悩み相談サイトに。そこに作成したスピーチ文を掲載し「これで大丈夫か添削してください」とメッセージを送ったというのである。「一晩で3人位の人が添削してくれましたよ。ちなみに自分以外にもそういう人はたくさんいて、その相談サイトには同じように添削をお願いする投稿がいっぱいありました」（由利さん）。ためしに教えてもらったそのサイトをみてみると、確かに「結婚式のスピーチの添削お願いします」という投稿が1000件を超えるくらいあった。めでたい結婚式の影でこんなネットコミュニティがあったとは知らなかった。しかし「事前にお願いされるスピーチなら準備ができるしまだいいですよ。一番嫌なのは何といってもサプライズスピーチ！」と語るのは昨年だけで10万以上お祝いにかかったというお祝い貧乏独女の薫さん（35歳）「私は基本的に人前で話すのが苦手なんですよ。だからいきなり指名されるとしどろもどろになって何もいえなくなる。そうすると自己嫌悪に陥って終わった後でもまったく楽しめなくなりますね」サプライズスピーチのメリットとしては、準備していない状態なので、フランクな本音をしゃべってもらえるという楽しさがあるようだ。しかしそれも上手に対応できる人ならいいが、苦手な人の場合だと「フランク」ではなく「しどろもどろ」になる危険性大。ちなみにプロの司会者の場合、本当のサプライズではなく式の最中に「のちほどサプライズスピーチとしてご指名させていただきま

### 各トピックの割合を確認
6600文章、8トピックのデータセットであるが、ほとんどのデータセットは13%～14%と同程度の割合で保存されていることがわかる。トピック5のみ、8%と他のトピックと比較してデータは少なめである。

In [6]:
alldata_original['label'].value_counts(9)

3   0.14
0   0.13
1   0.13
2   0.13
7   0.13
4   0.13
6   0.13
5   0.08
Name: label, dtype: float64

## 2.自然言語の前処理
自然言語の前処理として、正規表現による文字の除去、表記のゆれの統一、全角、半角の統一、ストップワードの設定、不要な言葉の削除を行う。
モデルの精度が十分で無い場合、以下の各前処理を工夫して精度向上を行う。

### 2.1.HTML表記の除去
今回は「Livedoor News」のデータセットを使用しているが、自然言語処理を行う場合、APIの活用、スクレイピングなどをしてデータを取得する場合があり、その場合は、分析に本質的に必要でないメタタグ（</A>など）を除去する必要がある。

In [7]:
#HTMLを除去

import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

alldata['text']=alldata['text'].apply(preprocessor)

### 2.2.表記ゆれの統一
本質的に同一の人、物、をさしているが表現が違っていることがある。（例：「山田君」、「山田さん」）これを「表記ゆれ」という。モデルを作るうえで、同じものをさしていることをモデルに学習させたいため、前処理として表現が違うが、同じものを表している場合、表現を統一する処理を行う。

In [8]:
#表記ゆれの統一
import re
def replace_yamada(text):
    replaced_text = re.sub(r'山田君', '山田さん', text)
    return replaced_text

#pandasaへ適用する場合
#targetという新しいカラムを作ることで元データを汚さないようにしています
alldata["text"] = alldata["text"].apply(replace_yamada)

### 2.3.全角、半角の統一
全角と半角は本質的には同じであるが、モデルは別のものと判断してしまうため、すべて全角、またはすべて半角などにして、表現を統一してやる。

In [9]:
#全角を半角に変換
import mojimoji

alldata['text']=alldata['text'].apply(mojimoji.zen_to_han)

### 2.4.数字の削除
トピック分類を行う上で数字が大きな意味を持たないと考えられる場合、最初に数値を削除する。

In [10]:
#数字をスペースに変換
def normalize_number(text):
    replaced_text = re.sub(r'\d+', '', text)
    return replaced_text
    
alldata["text"] = alldata["text"].apply(normalize_number) 

## 3.形態素解析
英語は、単語がスペースによって区切られているが、日本語はそのようなことはない。 
ここではMeCabを使用して、各文から名詞のみを取り出し、各文章がどのトピックに所属するか分析する。 
精度が十分で無い場合は、動詞、形容詞なども含めてモデルの作成を行う。

In [11]:
import MeCab
def wakati_by_mecab(text):
    tagger = MeCab.Tagger('')
    tagger.parse('') 
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        if pos in ["名詞"]:   # 対象とする品詞 ["名詞", "動詞", "形容詞"]
            word = node.surface
            word_list.append(word)
        node = node.next
    return " ".join(word_list)

alldata["text"] = alldata["text"].apply(wakati_by_mecab)

### 4.ストップワードの設定

In [12]:
#Stopword

import os
import urllib.request

def download_stopwords(path):
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(url, path)

def create_stopwords(file_path):
    stop_words = []
    for w in open(path, "r",encoding="utf-8_sig"):
        w = w.replace('\n','')
        if len(w) > 0:
            stop_words.append(w)
    return stop_words

In [13]:
path = "stop_words.txt"
download_stopwords(path)
stop_words = create_stopwords(path)

alldata["text"] = alldata["text"].apply(wakati_by_mecab)


File already exists.


### 5.単語文書行列の作成

### 5.1. CountVectorizer

In [14]:
#CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(stop_words=stop_words,token_pattern=u'(?u)\\b\\w+\\b')
feature_vectors_cnt = count_vectorizer.fit_transform(alldata["text"])
vocabulary_cnt = count_vectorizer.get_feature_names()

In [15]:
pd.DataFrame(feature_vectors_cnt.toarray(), columns=vocabulary_cnt)

,_,__,a,aa,aaa,aaaa,aac,aae,aaf,aandroid,...,ﾝﾀｯｸｽﾘｺｰｲﾒｰｼﾞﾝｸﾞﾎｰﾑﾍﾟｰｼﾞ,ﾝﾃﾞｽ,ﾝﾊ,ﾞｱｲﾝﾀｰﾅｼｮﾅﾙﾃﾞｺｽﾏｰﾄﾌｫﾝｶﾊﾞｰ,ﾞｼｬｰﾌﾟﾎｰﾑﾍﾟｰｼﾞｿﾆｰﾎｰﾑﾍﾟｰｼﾞ,ﾞﾃﾞｵｶﾒﾗﾌﾟﾛｼﾞｪｸﾀｰﾋﾞﾃﾞｵｶﾒﾗ,ﾟｭ,ﾟｯ,ﾟｰﾌﾟﾚｾﾞﾝﾄｷｬﾝﾍﾟｰﾝｲﾝﾓｰﾀﾙｽﾞ,ﾟｶﾄﾘｭﾌｧｽｾﾚｸｼｮﾝｵﾘｵｰﾙﾊﾞﾗｹﾞ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6600,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6601,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6602,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6603,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X_cnt = feature_vectors_cnt.toarray()
y_cnt = alldata['label']

In [17]:
%%time
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cnt, y_cnt, test_size=0.3, random_state=0, stratify=y_cnt)

Wall time: 1 s


In [18]:
%%time
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score

LGBMC=LGBMClassifier(random_state=0)
LGBMC.fit(X_train,y_train)
y_pred = LGBMC.predict(X_test)


scores = cross_val_score(estimator=LGBMC, X=X_train, y=y_train, cv=10, n_jobs=-1)

print("Accuracy(count_vectorizer): %.3f ± %.3f" % (np.mean(scores),np.std(scores)))

Accuracy(count_vectorizer): 0.946 ± 0.009
Wall time: 54.5 s


### 5.2. Tf-idf

In [19]:
#Tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
feature_vectors_Tfidf = tfidf.fit_transform(alldata["text"])
vocabulary_Tfidf = tfidf.get_feature_names() 

In [20]:
pd.DataFrame(feature_vectors_Tfidf.toarray(), columns=vocabulary_Tfidf)

,_,__,a,aa,aaa,aaaa,aac,aae,aaf,aandroid,...,ﾝﾀｯｸｽﾘｺｰｲﾒｰｼﾞﾝｸﾞﾎｰﾑﾍﾟｰｼﾞ,ﾝﾃﾞｽ,ﾝﾊ,ﾞｱｲﾝﾀｰﾅｼｮﾅﾙﾃﾞｺｽﾏｰﾄﾌｫﾝｶﾊﾞｰ,ﾞｼｬｰﾌﾟﾎｰﾑﾍﾟｰｼﾞｿﾆｰﾎｰﾑﾍﾟｰｼﾞ,ﾞﾃﾞｵｶﾒﾗﾌﾟﾛｼﾞｪｸﾀｰﾋﾞﾃﾞｵｶﾒﾗ,ﾟｭ,ﾟｯ,ﾟｰﾌﾟﾚｾﾞﾝﾄｷｬﾝﾍﾟｰﾝｲﾝﾓｰﾀﾙｽﾞ,ﾟｶﾄﾘｭﾌｧｽｾﾚｸｼｮﾝｵﾘｵｰﾙﾊﾞﾗｹﾞ
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6600,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6601,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6602,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6603,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [21]:
X_Tfidf = feature_vectors_Tfidf.toarray()
# X_Tfidf = feature_vectors_Tfidf.values
y_Tfidf = alldata['label'].values

In [22]:
%%time
from sklearn.model_selection import train_test_split
X_train_Tfidf, X_test_Tfidf, y_train_Tfidf, y_test_Tfidf = train_test_split(X_Tfidf, y_Tfidf, test_size=0.3, random_state=0, stratify=y_Tfidf)

Wall time: 987 ms


In [23]:
%%time
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score

LGBMC=LGBMClassifier(random_state=0)
LGBMC.fit(X_train_Tfidf,y_train_Tfidf)
y_pred = LGBMC.predict(X_test_Tfidf)

scores = cross_val_score(estimator=LGBMC, X=X_train_Tfidf, y=y_train_Tfidf, cv=10, n_jobs=-1)

print("Accuracy(count_Tfidf): %.3f ± %.3f" % (np.mean(scores),np.std(scores)))

Accuracy(count_Tfidf): 0.949 ± 0.010
Wall time: 1min 38s


In [24]:
y_pred_Tfidf = LGBMC.predict(X_test_Tfidf)

In [28]:
sentences = [token.split(" ") for token in alldata['text']]

In [29]:
%%time
# CBOWモデルの学習
from gensim.models import word2vec
cbow_model = word2vec.Word2Vec(sentences,
                                       sg=0,
                                       size=250,
                                       min_count=5,
                                       window=15,
                                       seed=1234)

Wall time: 9.11 s


In [30]:
# 作成したモデルの保存
cbow_model.save("Livedoor_cbow_w2v.model")
# saveしたモデルを読み込む時は
# model = word2vec.Word2Vec.load("./w2v.model")

In [31]:
# ここで記載しているscoreは、単語同士のコサイン類似度です。
pd.DataFrame(cbow_model.wv.most_similar(positive=['外食']), columns=["keyword", "score"])

,keyword,score
0,一人暮らし,0.88
1,自炊,0.87
2,財布,0.87
3,小銭,0.86
4,晩酌,0.86
5,夕食,0.86
6,発泡,0.85
7,ｸﾘｰﾆﾝｸﾞ,0.85
8,ﾗﾝﾁ,0.84
9,弁当,0.84


In [32]:
# skip-gramモデルの学習
skipgram_model = word2vec.Word2Vec(sentences,
                                               sg=1, 
                                               size=250,
                                               min_count=10,
                                               window=15, seed=1234)

In [33]:
# 作成したモデルの保存
skipgram_model.save("skipgram_w2v.model")
# saveしたモデルを読み込む時は
# model = word2vec.Word2Vec.load("./skipgram_w2v.model")

In [34]:
# 外食と似たキーワードを見つけていきます。ここで記載しているscoreは、単語同士のコサイン類似度です。
pd.DataFrame(skipgram_model.wv.most_similar(positive=['外食']), columns=["keyword", "score"])

,keyword,score
0,自炊,0.81
1,ご馳走,0.77
2,ﾗﾝﾁﾀｲﾑ,0.75
3,働き,0.73
4,夕飯,0.72
5,夕食,0.71
6,分担,0.71
7,やりくり,0.71
8,おかず,0.71
9,台所,0.70


In [37]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=LGBMC, X=X_train_Tfidf, y=y_train_Tfidf, cv=10, n_jobs=-1)

print("Accuracy: %.3f ± %.3f" % (np.mean(scores),np.std(scores)))

Accuracy: 0.949 ± 0.010


### 6.モデルの精度の確認

In [39]:
# Accuracy, Precision/Recall/F-score/Support, Confusion Matrix を表示
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer, accuracy_score

def show_evaluation_metrics(y_test, y_pred_Tfidf):
    print("Accuracy:")
    print(accuracy_score(y_test, y_pred))
    print()
    
    print("Report:")
    print(classification_report(y_test, y_pred))
    
    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred))

show_evaluation_metrics(y_test, y_pred)

Accuracy:
0.9566094853683148

Report:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       261
           1       0.98      0.97      0.97       261
           2       0.96      0.98      0.97       261
           3       0.97      0.99      0.98       271
           4       0.98      0.98      0.98       260
           5       0.95      0.80      0.87       154
           6       0.85      0.94      0.89       253
           7       1.00      0.99      0.99       261

    accuracy                           0.96      1982
   macro avg       0.96      0.95      0.95      1982
weighted avg       0.96      0.96      0.96      1982

Confusion matrix:
[[245   1   1   1   0   1  12   0]
 [  1 253   0   2   2   0   3   0]
 [  0   1 256   0   0   0   4   0]
 [  1   0   0 268   1   0   1   0]
 [  0   3   0   0 255   0   2   0]
 [  1   1   3   4   3 123  19   0]
 [  2   0   8   0   0   5 238   0]
 [  0   0   0   0   0   1   2 258]]


### 結論
今回は各文章から「名詞」だけを取り出して分類を行ったが、Accuracyが95.7%と非常に高い精度で分類が出来ることができた。
またコサイン類似度を測った結果、関係する単語を確認することが出来ており、ある程度想定通り動いていることが確認できた。

### Reference
- 機械学習・深層学習による自然言語処理入門(マイナビブックス)
- Python3×日本語：自然言語処理の前処理まとめ　https://qiita.com/chamao/items/7edaba62b120a660657e
- ニュース記事の分類を機械学習で予測する https://qiita.com/hyo_07/items/ba3d53868b2f55ed9941